# Batch Hydrolink 
## Developed By: Daniel Wieferich (USGS)
## 20171005

### Note: This code is a work in progress.  Any suggestions and comments are welcomed.

### This code performs a hydrolink batch process on a text file (.csv ) or shapefile (.shp) using user defined latitude and longitude fields.  The code returns the reachcode and measure of the closest position on the High Resolution National Hydrography Dataset and the Medium Resolution NHDPlusV2.1 using web services.  The code also uses snap distance and stream name to help quantify a level of certainty.  Levels of certainty currently exist only for the NHDPlusV2.1, due to the availability of needed information from current web services.

### Next steps: 
1. Work with the NHD High Resolution team to see if additional information can be returned from services to help with assigning a level of certainty. (in progress)
2. It would be great to get a measure of distance to closest confluence as a measure of certainty.  Through manual linkage in the past I've noticed that points within 50m of a confluence are more likely to link to the wrong reach.
3. The current code assumes lat/lon are in NAD83.  The code should be altered to accept various crs inputs.
4. Improve upon error checking and notifications
5. Allow for join of hydro-linked data to original dataset?

In [1]:
#The importFile function deals with importing csv or shps into a dataframe.  
def importFile (inputFileName, latFieldName, lonFieldName, streamFieldName, inputId):
    if inputFileName.endswith('.csv'):
        print ('\n' + 'reading csv file' +'\n')
        try:
            df = pd.read_csv(inputFileName)
        except KeyError:
            print ('file did not properly import, verify file name and rerun')
            #It would be nice here to reask for inputFileName and then restart at try statement
    elif inputFileName.endswith('.shp'):
        print('\n' + 'reading shapefile' + '\n')
        try:
            df = gp.GeoDataFrame.from_file(inputFileName)
        except KeyError:
            print ('file did not properly import, verify file name and rerun')
            #It would be nice here to reask for inputFileName and then restart at try statement
    else:
        print('File type not currently accepted. Please try .csv or .shp')
        
    if latFieldName in df and lonFieldName in df and streamFieldName in df and inputId in df:
        df = df[[inputId, latFieldName, lonFieldName, streamFieldName]].copy()
        df = df.rename(columns={inputId: 'id', latFieldName: 'lat', lonFieldName: 'lon', streamFieldName: 'stream'})
        return df
    else: 
        print ('verify field names and rerun')
        
#Using initial lat,lon (in nad83) to 
def hydrolinkHr(lat,lon,inputId):
    baseUrlHR = 'https://edits.nationalmap.gov/arcgis/rest/services/HEM/NHDHigh/MapServer/'
    getHrReach = baseUrlHR + 'exts/HEM_EditEvents_SOE/HEMGetReachcodeFromXY'
    getHrX = baseUrlHR + 'exts/HEM_EditEvents_SOE/HEMPointEvents'
    
    #Define variables, set initially as null values using similar denotation as the HydroLink Tool
    reachcodeHr = 'NO REACHCODE'
    measHigh = -999
    smDateHigh = ' '
    permId = 'NO PERM ID'
    xyHigh = '-999'
    
    #Structure original lat/lon into format needed for HEM SOE
    xy = '{"x":' + str(lon) + ',"y":' + str(lat) + ', "spatialReference": {"wkid":4269}}'
    
    if lat and (float(lat) > 24 and float(lat)<50) and lon and (float(lon) < -66 and float(lon)> -125):
        payload = {
            "point": xy ,
            "selectionLayerName": "NHDFLOWLINE",
            "selectionType": "TOPDOWNSTREAM",
            "searchToleranceMeters": 10000,
            "outWKID": 4269,
            "f": "json"}
        
        #Connects to web service
        r = requests.post(getHrReach,params=payload,verify=False).json()
        
        try:
            
                if r['resultStatus'] == 'success' and r['features']:
                    reachcodeHr = r['features'][0]['attributes']['REACHCODE']
                    payload2 = {
                      "point": xy ,
                      "reachcode": reachcodeHr,
                      "outWKID": 4269,
                      "f": "json"}

                    r2 = requests.post(getHrX,params=payload2,verify=False).json()
                    measHigh = r2['features'][0]['attributes']['MEASURE']
                    smDateHigh = r2['features'][0]['attributes']['REACHSMDATE']
                    permId = r2['features'][0]['attributes']['PERMANENT_IDENTIFIER']
                    xyHigh = r2['features'][0]['geometry']
                    return (reachcodeHr,measHigh,smDateHigh,permId,xyHigh)

                else:
                    return (reachcodeHr,measHigh,smDateHigh,permId,xyHigh)
        except:
            print ('Failed to process HR for: ' + str(inputId))
            return (reachcodeHr,measHigh,smDateHigh,permId,xyHigh)
    else:
        return (reachcodeHr,measHigh,smDateHigh,permId,xyHigh)

        
def hydrolinkMr(lat,lon,inputId):
    
    #Define variables, set initially as null values using similar denotation as the HydroLink Tool
    snapDist = -999
    xyMed = ''
    comidMed = 'NO COMID'
    reachMed = 'NO REACHCODE'
    measMed = -999
    gnisMed = ''
    
    #Additional Info about medium resolution service : https://www.epa.gov/waterdata/point-indexing-service
    xyMed = 'POINT(' + str(lon) + ' ' + str(lat) + ')'
    
    payloadMr = {
        'optNHDPlusDataset': '2.1',
        'pGeometry': xyMed,
        'pGeometryMod': 'SRID=8265',
        'pOutputPathFlag': 'TRUE',
        'pPointIndexingMaxDist': '2', #Kilometers
        'pPointIndexingMethod':'DISTANCE',
        'pResolution': '3',
        'pPointIndexingFcodeDeny' : [56600],
        'pReturnFlowlineGeomFlag':'FALSE',
        'f':'json'}
    
    baseUrlMr = 'https://ofmpub.epa.gov/waters10/PointIndexing.Service'
    
    #Connects to web service
    rMed = requests.post(baseUrlMr,params=payloadMr, verify=False).json()
    
    try:
        snapDist = rMed['output']['total_distance']
        xyMed = rMed['output']['end_point']['coordinates']
        comidMed = rMed['output']['ary_flowlines'][0]['comid']
        reachMed = rMed['output']['ary_flowlines'][0]['reachcode']
        measMed = rMed['output']['ary_flowlines'][0]['fmeasure']
        gnisMed = rMed['output']['ary_flowlines'][0]['gnis_name']
        return (snapDist,xyMed,comidMed,reachMed,measMed,gnisMed)
       
    except:
        print 
        return (snapDist,xyMed,comidMed,reachMed,measMed,gnisMed)
    

def cleanStreamName(stream):
        #remove case for case sensitive operations
        stream = stream.lower()
        
        #replace common abbreviations, this needs improvement but be careful not to replace strings we dont want to
        #this code currently assumes GNIS_NAME never contains abbreviations... something to verify
        #If you have a better way to do this let me know!!!!
        stream = stream.replace(' st ', 'stream')
        stream = stream.replace(' st.', 'stream')
        stream = stream.replace(' rv ', 'river')
        stream = stream.replace(' rv.', 'river')
        stream = stream.replace(' trib.', 'tributary')
        stream = stream.replace(' trib)', 'tributary')
        stream = stream.replace(' trib ', 'tributary')
        stream = stream.replace(' ck ', 'creek')
        stream = stream.replace(' ck.', 'creek')
        stream = stream.replace(' br ', 'branch')
        stream = stream.replace(' br.', 'branch')
        
        return stream
                       
def mrCertainty(streamClean, gnisMed):
    import difflib
    
    
    #Stream Name Match
    if streamClean and gnisMed:
        gnisMed = gnisMed.lower()
        #print ('\n' + gnisMed)
        #print (streamClean)

        if gnisMed == streamClean:
            gnisCertMr = 1
            #print (gnisCert)
        #do not want name of main stem to be fuzzy matched.  To avoid remove those names with tributary or branch in them
        elif 'tributary' in streamClean or 'branch' in streamClean:
            gnisCertMr = 0
            #print (gnisCert)
        else:
            matchRatio = difflib.SequenceMatcher(lambda x: x == " ",streamClean, gnisMed).ratio()
            
            # From Python Documentation (https://docs.python.org/3/library/difflib.html):
            #"As a rule of thumb, a ratio() value over 0.6 means the sequences are close matches:"
            # At some point we should validate this rule of thumb but figured this is a good starting place
            if matchRatio >= 0.6:
                gnisCertMr = 1
                #print (gnisCert)
            #this is likely a match but less certain
            elif 0.6 > matchRatio >= 0.5:
                gnisCertMr = 0.5
                #print (gnisCert)
            else:
                gnisCertMr = 0
                #print (gnisCert)
                
    #else no stream name is supplied for one or both datasets, therefor stream name does not help improve certainty
    else:
        gnisCertMr = 0
    #--------------------------------------------------------------------   
    #Distance between original lat/lon and reach/measure linkage
    #Note: snapDist is returned in Kilometers
    if snapDist:
        #print (snapDist)
        if snapDist == -999:
            distCertMr = 0
            #print (distCertMr)
        elif snapDist <=0.050:
            distCertMr = 1
            #print (distCertMr)
        elif 0.200<= snapDist <0.050:
            distCertMr = 0.5
            #print (distCertMr)
        else:
            distCertMr = 0
            #print (distCertMr)
    else:
        distCertMr = 0
        
        
    #ADD CODE TO CHECK DISTANCE TO CLOSEST CONFLUENCE, if less than ?50m? flag as needs visual confirmation
           
        
    return gnisCertMr, distCertMr
        

In [2]:
import requests
import json
import geopandas as gp
import pandas as pd
import warnings

warnings.filterwarnings("ignore")

#User inputs variables needed for code to run
inputFileName = input("Enter file name, including extension (only accepts .csv and .shp): ")
latFieldName = input("Enter field name for latitude, note this is case sensitive: ")
lonFieldName = input("Enter field name for longitude, note this is case sensitive: ")
streamFieldName = input("Enter field name for stream name, note this is case sensitive: ")
inputId = input("Enter field name for identifier, note this is case sensitive: ")

#Alternative to user input, info can be hard coded here
#inputFileName = 'testData/test2.shp'
#latFieldName = 'DamLatitud'
#lonFieldName = 'DamLongitu'
#streamFieldName = 'DamRiverNa'
#inputId = 'DamName'


outData = []

df = importFile (inputFileName, latFieldName, lonFieldName, streamFieldName, inputId)

for row in df.itertuples():
        
        #Define variables based on row,field values
        lat = row.lat
        lon = row.lon
        inputId = row.id
        stream = row.stream
        
        print ('working on : ' + str(inputId))
                
        reachcodeHr,measHigh,smDateHigh,permId,xyHigh = hydrolinkHr(lat,lon,inputId)
        snapDist,xyMed,comidMed,reachMed,measMed,gnisMed = hydrolinkMr(lat,lon,inputId)
        streamClean = cleanStreamName(stream)
        gnisCertMr, distCertMr = mrCertainty(streamClean, gnisMed)
        
        #record data in outData, this will be used to create dataframe
        outData.append({"id":inputId,"ReachHigh":reachcodeHr,"ReachMed":reachMed,"MeasHigh":measHigh,"MeasMed":measMed,"SmdateHigh":smDateHigh, "Comid":comidMed, "permIdHr": permId, "xyHigh":xyHigh, "xyMed":xyMed,"gnisCertMr":gnisCertMr, "distCertMr":distCertMr })

#Create Dataframe with hydro-link data        
outDf = pd.DataFrame(outData)

Enter file name, including extension (only accepts .csv and .shp): testData/test2.shp
Enter field name for latitude, note this is case sensitive: DamLatitud
Enter field name for longitude, note this is case sensitive: DamLongitu
Enter field name for stream name, note this is case sensitive: DamRiverNa
Enter field name for identifier, note this is case sensitive: DamName

reading shapefile

working on : New River Light and Power Dam
working on : Fifth Ave Dam
working on : Homestead Dam
working on : Kentchurch Dam
working on : Main Street Dam
working on : Marvel Slab Dam
working on : Off-Billington Street Dam
working on : Shull's Mill Dam
working on : Lower Flume Dam
working on : Fell Spice Mill Dam
working on : Upper Flume Dam
working on : Great Works Dam
working on : Hemlock Dam
working on : Thompson Dam
working on : Mendaraz Dam
working on : Spruce Pine Dam
working on : Pelham Dam (Bartlett Fishrod Company Dam)
working on : Birch Run Dam
working on : San Clemente Dam
working on : Veaz

In [3]:
outDf

,Comid,MeasHigh,MeasMed,ReachHigh,ReachMed,SmdateHigh,distCertMr,gnisCertMr,id,permIdHr,xyHigh,xyMed
0,6892046,14.16452,12.98281,05050001000438,05050001000438,1330560000000,1,1,New River Light and Power Dam,{4A50F06E-D38A-4818-AF2B-1C8C24762D4B},"{'x': -81.64585508393577, 'y': 36.209041733997...","[-81.6461351020147, 36.2090409025584]"
1,5218153,-999.00000,43.66452,NO REACHCODE,05060001000211,,1,1,Fifth Ave Dam,NO PERM ID,-999,"[-83.0244554424495, 39.9887805191651]"
2,6778181,38.61249,38.59254,01080201000081,01080201000081,936576000000,1,1,Homestead Dam,{891B43ED-B9EF-4DFD-BFF2-02AC32ADF36B},"{'x': -72.32807209393806, 'y': 42.871675818880...","[-72.3280017934781, 42.8716649453981]"
3,NO COMID,-999.00000,-999.00000,NO REACHCODE,NO REACHCODE,,0,0,Kentchurch Dam,NO PERM ID,-999,POINT(-2.865927 51.92675)
4,5218161,84.81290,78.34369,05060001005352,05060001002526,1482240294000,1,1,Main Street Dam,{B0921889-0691-45DA-B810-DB7F86B34E73},"{'x': -83.00753190217114, 'y': 39.955464878369...","[-83.0076984876497, 39.9556309896144]"
5,21658128,61.88451,52.65591,03150202003380,03150202003380,1330646400000,1,1,Marvel Slab Dam,{D2342D4B-C7FE-43D1-93B2-0FF7B3862502},"{'x': -87.02958802287841, 'y': 33.165084553097...","[-87.0293324538523, 33.1651026027221]"
6,5878571,-999.00000,53.61827,NO REACHCODE,01090002001624,,1,1,Off-Billington Street Dam,NO PERM ID,-999,"[-70.6737186376754, 41.9489482229312]"
7,19743560,-999.00000,73.13481,NO REACHCODE,06010103000155,,1,1,Shull's Mill Dam,NO PERM ID,-999,"[-81.7470035031778, 36.1831559685481]"
8,NO COMID,24.11306,-999.00000,01090002025274,NO REACHCODE,1331078400000,0,0,Lower Flume Dam,{AE6D95EF-2324-424B-9F39-463DC0A0CCBC},"{'x': -70.63404996055921, 'y': 41.765271487451...",POINT(-70.634066 41.765246)
9,4651912,21.13665,21.38402,02040205000108,02040205000108,1331164800000,1,1,Fell Spice Mill Dam,{42882348-740F-446B-9477-EE2AD5D18519},"{'x': -75.63284287436163, 'y': 39.750183539320...","[-75.6327053822628, 39.7500695852434]"


In [4]:
outFile = input("Name of output file. No extension:  ")
outFile = outFile + ".csv"
outDf.to_csv(outFile)

Name of output file. No extension:  test2Out
